In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import re

train = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\train.csv\train.csv")
test  = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\test.csv\test.csv")

train1 = train.drop(['Descript', 'Resolution'], axis = 1)

train1 = train1.query('Y != 90')

In [2]:
Dt = pd.to_datetime(train1['Dates'])
train1['Date'] = Dt.dt.date
train1['Year'] = Dt.dt.year
train1['Month'] = Dt.dt.month
train1['Day'] = Dt.dt.day
train1['Hour'] = Dt.dt.hour
train1.drop(["Dates"], axis = 1, inplace = True)

le = LabelEncoder()
train1['DayOfWeek'] = le.fit_transform(train1['DayOfWeek'])

count_encoding = train1['PdDistrict'].value_counts().to_dict()
train1['PdDistrict'] = train1['PdDistrict'].map(count_encoding)

Ad_dict = {'.* / .*':0, '.*':1}
train1['Address'] = train1['Address'].replace(Ad_dict,regex = True)


train1['Category'] = le.fit_transform(train1['Category'])

train1.head()
# # DayOfWeek...label, PdDistrict...count

,Category,DayOfWeek,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour
0,37,6,105284,0,-122.425892,37.774599,2015-05-13,2015,5,13,23
1,21,6,105284,0,-122.425892,37.774599,2015-05-13,2015,5,13,23
2,21,6,105284,0,-122.424363,37.800414,2015-05-13,2015,5,13,23
3,16,6,105284,1,-122.426995,37.800873,2015-05-13,2015,5,13,23
4,16,6,49311,1,-122.438738,37.771541,2015-05-13,2015,5,13,23


In [3]:
Dt = pd.to_datetime(test['Dates'])
test['Date'] = Dt.dt.date
test['Year'] = Dt.dt.year
test['Month'] = Dt.dt.month
test['Day'] = Dt.dt.day
test['Hour'] = Dt.dt.hour
test.drop(["Dates"], axis = 1, inplace = True)

le = LabelEncoder()
test['DayOfWeek'] = le.fit_transform(test['DayOfWeek'])

count_encoding = test['PdDistrict'].value_counts().to_dict()
test['PdDistrict'] = test['PdDistrict'].map(count_encoding)

Ad_dict = {'.* / .*':0, '.*':1}
test['Address'] = test['Address'].replace(Ad_dict,regex = True)
test.head()

,Id,DayOfWeek,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour
0,0,3,89591,1,-122.399588,37.735051,2015-05-10,2015,5,10,23
1,1,3,89591,0,-122.391523,37.732432,2015-05-10,2015,5,10,23
2,2,3,107017,1,-122.426002,37.792212,2015-05-10,2015,5,10,23
3,3,3,80084,1,-122.437394,37.721412,2015-05-10,2015,5,10,23
4,4,3,80084,1,-122.437394,37.721412,2015-05-10,2015,5,10,23


In [4]:
cols = ['DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour']

X = train1[cols]
Y = train1['Category']

X_test = test[cols]

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.3, random_state = 1219)

In [5]:
from catboost import CatBoostClassifier
cb = CatBoostClassifier()

cb.fit(X_train, y_train)
y_pred = cb.predict(X_val)

Learning rate set to 0.109784
0:	learn: 3.3030947	total: 1.66s	remaining: 27m 39s
1:	learn: 3.1333088	total: 3.07s	remaining: 25m 30s
2:	learn: 3.0182566	total: 4.34s	remaining: 24m 1s
3:	learn: 2.9289445	total: 5.77s	remaining: 23m 56s
4:	learn: 2.8614071	total: 7.1s	remaining: 23m 32s
5:	learn: 2.8057281	total: 8.4s	remaining: 23m 12s
6:	learn: 2.7600281	total: 9.72s	remaining: 22m 59s
7:	learn: 2.7209540	total: 11s	remaining: 22m 49s
8:	learn: 2.6884995	total: 12.4s	remaining: 22m 44s
9:	learn: 2.6608813	total: 13.7s	remaining: 22m 36s
10:	learn: 2.6389979	total: 14.9s	remaining: 22m 20s
11:	learn: 2.6190923	total: 16.2s	remaining: 22m 9s
12:	learn: 2.6017114	total: 17.4s	remaining: 22m
13:	learn: 2.5856929	total: 18.6s	remaining: 21m 49s
14:	learn: 2.5726743	total: 19.9s	remaining: 21m 47s
15:	learn: 2.5615021	total: 21.2s	remaining: 21m 43s
16:	learn: 2.5520595	total: 22.5s	remaining: 21m 42s
17:	learn: 2.5423675	total: 23.9s	remaining: 21m 45s
18:	learn: 2.5336274	total: 25.2s	re

In [6]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val, y_pred)

0.28691888608363864

In [7]:
cb_submission = cb.predict(X_test)

In [ ]:
import csv
pred = cb.predict_proba(test[['DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour']])
submission = pd.DataFrame(columns=['Id'], data=test)
submission = pd.concat([submission, pd.DataFrame(pred, columns=["ARSON", "ASSAULT", "BAD CHECKS", "BRIBERY", "BURGLARY", "DISORDERLY CONDUCT",
"DRIVING UNDER THE INFLUENCE", "DRUG/NARCOTIC", "DRUNKENNESS", "EMBEZZLEMENT", "EXTORTION",
"FAMILY OFFENSES", "FORGERY/COUNTERFEITING", "FRAUD", "GAMBLING", "KIDNAPPING", "LARCENY/THEFT",
"LIQUOR LAWS", "LOITERING", "MISSING PERSON", "NON-CRIMINAL", "OTHER OFFENSES", "PORNOGRAPHY/OBSCENE MAT",
"PROSTITUTION", "RECOVERED VEHICLE", "ROBBERY", "RUNAWAY", "SECONDARY CODES", "SEX OFFENSES FORCIBLE",
"SEX OFFENSES NON FORCIBLE", "STOLEN PROPERTY", "SUICIDE", "SUSPICIOUS OCC", "TREA", "TRESPASS",
"VANDALISM", "VEHICLE THEFT", "WARRANTS", "WEAPON LAWS"])], axis=1)

In [ ]:
submission.to_csv("cat1.csv", index=False)